## First approach: Neural Network
The MLP Regressor model of sklearn library is employed.

#### Importing libraries and datasets

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
#The usual collection of indispensables 
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
import scipy.fftpack
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error, explained_variance_score, max_error
from sklearn.model_selection import cross_val_score, train_test_split

In [2]:
from helpers import sample_data, load_data

# load data.
data_oligo_1 = np.delete(load_data("data/data-oligo/011021_SFL_SYN211_Oligo_1uM_Rawdata_270spectralcolumns.csv"), 0, 1).T
data_oligo_1 = data_oligo_1[data_oligo_1[:, 0] != -999, :]
data_oligo_2 = np.delete(load_data("data/data-oligo/051021_SFL_SYN211_Oligo_5uM_rawdata_270spectralcolumns.csv"), 0, 1).T
data_oligo_2 = data_oligo_2[data_oligo_2[:, 0] != -999, :]

data_oligo = np.append(data_oligo_1, data_oligo_2, axis = 0)

y_oligo = np.expand_dims(np.zeros(len(data_oligo)), axis=1)

data_PFF1 = np.delete(load_data("data/data-pff/191121_G80_AInII_SYn211_AsynPFF_5microM_rawdata_290spectracolumns.csv"), 0, 1).T
data_PFF1 = data_PFF1[data_PFF1[:, 0] != -999, :]
data_PFF2 = np.delete(load_data("data/data-pff/220421_G80_AInII_SYn211_AsynPFF_20microM_880_spectralcolumns.csv"), 0, 1).T
data_PFF2 = data_PFF2[data_PFF2[:, 0] != -999, :]

data_PFF =  np.append(data_PFF1, data_PFF2, axis = 0)

y_PFF = np.expand_dims(np.ones(len(data_PFF)), axis=1)

mix_50_50 = np.delete(load_data("data/data-mix/1221_G80_AI_SYn211_2uMPFF50__2uMOligo50__rawdata_840spectralcolumns.csv"), 0, 1).T
mix_50_50 = mix_50_50[mix_50_50[:, 0] != -999, :]

y_50_50 = np.expand_dims(np.ones(len(data_PFF))*.5, axis=1)

mix_75_25 = np.delete(load_data("data/data-mix/1221_G80_AI_SYn211_4.5uMPFF75__1.5uMOligo25__Rawdata_710spectralcolumns.csv"), 0, 1).T
mix_75_25 = mix_75_25[mix_75_25[:, 0] != -999, :]

y_75_25 = np.expand_dims(np.ones(len(mix_75_25))*.75, axis=1)

mix_25_75 = np.delete(load_data("data/data-mix/1221_G80_AI_SYn211_4.5uMOligo75%_1.5uMPFF25%_Rawdata_730spectralcolumns.csv"), 0, 1).T
mix_25_75 = mix_25_75[mix_25_75[:, 0] != -999, :]

y_25_75 = np.expand_dims(np.ones(len(mix_25_75))*.25, axis=1)

# Load the new datasets
mix_60_40 = np.delete(load_data("data/data-mix/1221_G80_AI_SYn211_2.4uMOligo40%_3.6uMPFF60%_Rawdata_410spectralcolumns.csv"), 0, 1).T
mix_60_40 = mix_60_40[mix_60_40[:, 0] != -999, :]
y_60_40 = np.expand_dims(np.ones(len(mix_60_40))*0.6, axis=1)

mix_40_60 = np.delete(load_data("data/data-mix/1221_G80_AI_SYn211_2.4uMOligo60%_1.6uMPFF40%_Rawdata_350spectralcolumns.csv"), 0, 1).T
mix_40_60 = mix_40_60[mix_40_60[:, 0] != -999, :]
y_40_60 = np.expand_dims(np.ones(len(mix_40_60))*0.4, axis=1)


print(data_oligo.shape, data_PFF.shape, mix_50_50.shape, mix_75_25.shape, mix_25_75.shape, mix_40_60.shape, mix_60_40.shape)


(540, 133) (1170, 133) (840, 133) (710, 133) (730, 133) (350, 133) (410, 133)


#### Show improvement in regression when adding data
First, we'll train the model, on 0-100, 100-0 and test on dataset of 40-60 and 60-40.

In [73]:
#Build X and y by concatenating the different dataset (after sampling the same number of datapoints for each dataset) 
np.random.seed()
X = np.concatenate((data_oligo[np.random.randint(data_oligo.shape[0], size = 350), :], data_PFF[np.random.randint(data_PFF.shape[0], size=350),:]), axis = 0)
y = np.concatenate((y_oligo[np.random.randint(y_oligo.shape[0], size = 350), :], y_PFF[np.random.randint(y_PFF.shape[0], size=350),:]), axis = 0)

X_test = np.concatenate((mix_40_60[np.random.randint(mix_40_60.shape[0], size=350),:],mix_60_40[np.random.randint(mix_60_40.shape[0], size=350),:]), axis = 0)
y_test = np.concatenate((y_40_60[np.random.randint(y_40_60.shape[0], size=350),:],y_60_40[np.random.randint(y_60_40.shape[0], size=350),:]), axis = 0)
indices = [np.random.choice(700, 200) for i in range(100)]

x_test, y_test = [X_test[idx] for idx in indices], [y_test[idx] for idx in indices]
X_train, _, y_train, _ = train_test_split( X, y, random_state=42)

In [74]:
def calculate_predictions_scores(model, x_test, y_test):
    predictions = []
    mses = []
    maes = []
    mxs = []
    for i in range(len(x_test)):
        predictions.append(model.predict(x_test[i]))
        mses.append(mean_squared_error(y_test[i], predictions[i]))
        maes.append(mean_absolute_error(y_test[i], predictions[i]))
        mxs.append(max_error(y_test[i], predictions[i]))
    return np.array(mses), np.array(maes), np.array(mxs)

In [75]:
# Training the neural network
model = MLPRegressor(solver='adam', activation='logistic', alpha=1e-5, hidden_layer_sizes=(10,10,10,10), random_state=1)
model.fit(X_train, y_train.squeeze())

# Predicting the 50-50 mix
mses, maes, mxs = calculate_predictions_scores(model, x_test, y_test)
print(f"Mean squared error: {mses.mean():.4f}\nMean absolute error: {maes.mean():.4f}\nMax Error: {mxs.mean():.4f}")

Mean squared error: 0.1913
Mean absolute error: 0.4199
Max Error: 0.5785


Now we add 50-50 to the train set and repeat the testing on the 40-60 / 60-40 dataset.

In [76]:
#Build X and y by concatenating the different dataset (after sampling the same number of datapoints for each dataset) cv 1 fold
#np.random.seed()
X = np.concatenate((data_oligo[np.random.randint(data_oligo.shape[0], size = 350), :], data_PFF[np.random.randint(data_PFF.shape[0], size=350),:],mix_50_50[np.random.randint(mix_50_50.shape[0], size=350),:]), axis = 0)
y = np.concatenate((y_oligo[np.random.randint(y_oligo.shape[0], size = 350), :], y_PFF[np.random.randint(y_PFF.shape[0], size=350),:],y_50_50[np.random.randint(y_50_50.shape[0], size=350),:]), axis = 0)

X_train, _, y_train, _ = train_test_split( X, y, random_state=42)

In [77]:
# Training the neural network
model = MLPRegressor(solver='adam', activation='logistic', alpha=1e-5, hidden_layer_sizes=(10,10,10,10), random_state=1)
model.fit(X_train, y_train.squeeze())
# Predicting the 25-75 mix
mses, maes, mxs = calculate_predictions_scores(model, x_test, y_test)
print(f"Mean squared error: {mses.mean():.4f}\nMean absolute error: {maes.mean():.4f}\nMax Error: {mxs.mean():.4f}")

Mean squared error: 0.0572
Mean absolute error: 0.2111
Max Error: 0.5360


As we can see, both mean squared error and mean absolute error decrease.

Now we train on percentages 25-75 and 75-25 and repeat the testing on the 40-60 / 60-40 dataset.

In [78]:
#Build X and y by concatenating the different dataset (after sampling the same number of datapoints for each dataset) cv 1 fold
#np.random.seed()
X = np.concatenate((data_oligo[np.random.randint(data_oligo.shape[0], size = 350), :], data_PFF[np.random.randint(data_PFF.shape[0], size=350),:],mix_50_50[np.random.randint(mix_50_50.shape[0], size=350),:], mix_25_75[np.random.randint(mix_25_75.shape[0], size=350),:], mix_75_25[np.random.randint(mix_75_25.shape[0], size=350),:]), axis = 0)
y = np.concatenate((y_oligo[np.random.randint(y_oligo.shape[0], size = 350), :], y_PFF[np.random.randint(y_PFF.shape[0], size=350),:],y_50_50[np.random.randint(y_50_50.shape[0], size=350),:], y_75_25[np.random.randint(y_75_25.shape[0], size=350),:], y_25_75[np.random.randint(y_25_75.shape[0], size=350),:]), axis = 0)
# Train and test

X_train, _, y_train, _ = train_test_split( X, y, random_state=42)

In [79]:
# Training the neural network
model = MLPRegressor(solver='adam', activation='logistic', alpha=1e-5, hidden_layer_sizes=(10,10,10,10), random_state=1)
model.fit(X_train, y_train.squeeze())
# Predicting the 40-60
mses, maes, mxs = calculate_predictions_scores(model, x_test, y_test)
print(f"Mean squared error: {mses.mean():.4f}\nMean absolute error: {maes.mean():.4f}\nMax Error: {mxs.mean():.4f}")

Mean squared error: 0.0306
Mean absolute error: 0.1391
Max Error: 0.4179


#### Perform parameter tuning
The data regarding the 5 different mixes (0-100, 25-75, 50-50, 75-25, 100-0) is combined.
Then, this data is split, 80% will be used for training and 20% for testing.

In [3]:
#Build X and y by concatenating the different dataset (after sampling the same number of datapoints for each dataset) cv 1 fold
np.random.seed()
X = np.concatenate((data_oligo, data_PFF[np.random.randint(data_PFF.shape[0], size=540),:],mix_50_50[np.random.randint(mix_50_50.shape[0], size=540),:], mix_75_25[np.random.randint(mix_75_25.shape[0], size=540),:], mix_25_75[np.random.randint(mix_25_75.shape[0], size=540),:], mix_60_40[np.random.randint(mix_60_40.shape[0], size = 350), :], mix_40_60[np.random.randint(mix_40_60.shape[0], size = 350), :]), axis = 0)
y = np.concatenate((y_oligo, y_PFF[np.random.randint(y_PFF.shape[0], size=540),:],y_50_50[np.random.randint(y_50_50.shape[0], size=540),:], y_75_25[np.random.randint(y_75_25.shape[0], size=540),:], y_25_75[np.random.randint(y_25_75.shape[0], size=540),:], y_60_40[np.random.randint(y_60_40.shape[0], size = 350), :], y_40_60[np.random.randint(y_40_60.shape[0], size = 350), :]), axis = 0)
# 80-20 Train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

The model is trained and tested to determine the values of height, depth and alpha which yield the best result (can require some minutes). 

In [4]:
# Training the neural network with parameter tuning, minimizing mean absolute error
depth = [2*i for i in range(1,21)] #range of depth, can be changed  
height = [2*i for i in range(1,21)] #range of heights, can be changed
alphas = [1e-3, 1e-4, 1e-5] #values of alphas, can be changed
track_mae = []
best_mae = []
track_mse = []
best_mse = []
for d in tqdm(depth):
    for h in tqdm(height):
        layers = tuple([h for i in range(d)])
        for a in alphas:
            model = MLPRegressor(solver='adam', activation='logistic', alpha=a, hidden_layer_sizes=layers, max_iter=500)
            mae_scores = - cross_val_score(model, X_train, y_train.squeeze(), cv=5, scoring = "neg_mean_absolute_error")
            track_mae.append([mae_scores.mean(), layers, a])
            if (len(track_mae) == 1) or (len(track_mae)>1 and track_mae[-1][0] < best_mae[0]):
                best_mae = track_mae[-1]
            mse_scores = - cross_val_score(model, X_train, y_train.squeeze(), cv=5, scoring = "neg_mean_squared_error")
            track_mse.append([mse_scores.mean(), layers, a])
            if (len(track_mse) == 1) or (len(track_mse)>1 and track_mse[-1][0] < best_mse[0]):
                best_mse = track_mse[-1]

print(end = '\n')

            

100%|██████████| 20/20 [12:24:18<00:00, 2232.95s/it]

In [5]:
print(f"Best Mean absolute error: {best_mae[0]:.4f}. Height: {best_mae[1][0]}. Depth: {len(best_mae[1])}. Alpha: {best_mae[2]}")
print(f"Best Mean squared error: {best_mse[0]:.4f}. Height: {best_mse[1][0]}. Depth: {len(best_mse[1])}. Alpha: {best_mse[2]}")


Best Mean absolute error: 0.0612. Height: 36. Depth: 6. Alpha: 1e-05
Best Mean squared error: 0.0141. Height: 36. Depth: 6. Alpha: 1e-05
